# Methods

## 1. Loading, Wrangling, and Cleaning the Data

In [ ]:
#loading in the library necessary
library(tidyverse)
install.packages("tidyverse")
library(repr)
library(tidymodels)
install.packages("themis")
library(themis)

#formatting
options(repr.matrix.max.rows = 6)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes     

Our data analysis was performed using a classification model, where a positive or negative diagnosis of heart disease in a patient was predicted based on the individual's various attributes. We selected predictors based on expected relevance to our thesis, which was to explore the correlation between the dataset’s given attributes and coronary artery disease (CAD).

The original dataset contains the following attributes: 

1. Age in years (__age__)
> The ages in the dataset roughly range from early 30s - late 70s. 

2. __sex__ - EXCLUDED

3. Type of chest pain (__cp__)
> Angina is a type of chronic chest pain that occurs when the heart is deficient in oxygen, and is a common symptom of cardiac disease. The original dataset distinguishes chest pain into 4 classes:
> * __*angina*__ = typical angina, where the patient experiences normal anginal symptoms
> * __*abnang*__ = atypical angina, where the patient experiences abnormal anginal symptoms
> * __*notang*__ = non-anginal pain, where the patient experiences chest pain that is unrelated to oxygen deficiency in the heart
> * __*asympt*__ = asymptomatic, where the patient experiences no chest pain at all

4. Resting blood pressure in mmHg (__trestbps__)
> *Note*: a normal resting blood pressure value is 120/80 mmHg (simplified to 120 mmHg to fit the original data), or slightly below.

5. Serum cholesterol in mg/dL (__chol__)
> Serum cholesterol is the total amount of cholesterol in the blood. Over time, cholesterol build up can lead to vascular blockage, which often occurs in the arteries and is a leading factor in CAD.
> 
> *Note*: An average "healthy" serum cholesterol level is conventionally taken to be below 200 mg/dL (5.5 mmol/L). However, serum cholesterol is made up of different types of cholesterol, some of which are actually indicative of individual health and help to eliminate cholesterol build up. The dataset only provides values for total cholesterol levels; thus, we only analyze the relationship between _total_ cholesterol and CAD.

6. Fasting blood sugar in mg/dL (__fbs__)
> According to the World Health Organization (WHO), normal fasting blood sugar is expected to be between 70 and 100 mg/dL. Having a fasting blood sugar level of 100 mg/dL or higher is an indicator of prediabetes - a condition where insulin is increasingly overproduced in the body and puts the patient at a high risk of developing type 2 diabetes, cardiac diseases and stroke; type 2 diabetes is confirmed at a fasting blood sugar level of at least 125 mg/dL. The researchers of this dataset have categorized patients as either:
> * subceeding (__*fal*__) 120 mg/dL
> * exceeding (__*true*__) 120 mg/dL.

7. __restecg__ - EXCLUDED

8. Maximum heart rate achieved in bpm (__thalach__)
> Values were taken at time of hospital admission.
>
> *Note*: According to the Centers for Disease Control and Prevention (CDC), the expected maximum heart rate of an adult individual is generally taken to be the individual's age in years subtracted from 220 (220 - age). For our model, we analyzed the observed maximum heart rates based on relative deviation from each individual’s theoretical maximum heart rate.

9. __exang__ - EXCLUDED

10. __oldpeak__ - EXCLUDED

11. __slope__ - EXCLUDED

12. Number of major vessels imaged by fluoroscopy (__ca__)
> Fluoroscopy is a radiation technique that is often used to scan for vascular blockage in the heart, a common factor in CAD. The number of major vessels imaged by fluoroscopy is given as a range of 0-3.
> 
> _Note:_ It is not mentioned the specific vessels that were inspected for, however, we expect that they include the coronary arteries, which are the most common sites of vascular blockage in heart disease.

14. __thal__ - EXCLUDED
<!-- > Thalassemia is a genetic disorder characterized by insufficient hemoglobin production in the body, which reduces red blood cell longevity and efficient delivery of oxygen to the tissues via the blood. 
> * __*norm*__ = normal
> * __*fixed*__ = fixed defect
> * __*rever*__ = reversable defect -->

14. Diagnosis of CAD (__num__)
> _Note_: The dataset does not elaborate much on our classifier column of interest, only mentioning the classification of “heart disease”. “Heart disease” is a broad term for conditions that affect any part of the cardiovascular system; however, it is also interchangeably used with coronary artery disease (CAD), a disorder affiliated with the coronary arteries that commonly develops later in life. As the dataset exhibits a relatively late age range (early 30s - late 70s) and investigates attributes that seem to be directly involved with the heart, we have assumed that the dataset defines "heart disease" as CAD.
> 
> CAD diagnoses are classified as:
> * __*buff*__ = negative (not heart disease)
> * __*sick*__ = positive (confirmed heart disease)

The classifier we chose was the attribute __num__, which classifies patients as testing negative (__buff__) or positive (__sick__) for heart disease. The variables we chose for our model predictors were age (__age__), type of chest pain (__cp__), resting blood pressure (__trestbps__; mmHg), serum cholesterol (__chol__; mg/dL), fasting blood sugar (__fbs__; mg/dL), observed maximum heart rate (__thalach__; bpm), and the number of major vessels that exhibited vascular blockage for each patient (__ca__; 0-3). The remaining attributes (__sex, restecg, exang, oldpeak, slope,__ and __thal__) were excluded from our model due to perceived irrelevance and/or insufficient information for interpretation, and thus are not elaborated on in our report. We also removed the last column in the original dataset as no background information was provided for it. 

In [ ]:
#loading the data
cleveland <- read_csv("https://raw.githubusercontent.com/JennWan/Group_Project/main/new%20data/newcleveland_data.csv", col_names = F, show_col_types = F)

#renaming variables for readability 
colnames(cleveland) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                         "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

#removing 15th column (NA)
cleveland <- cleveland |>
                    select(!15)

print("Table 1.1: Original Cleveland dataset (with headers)")
cleveland

In [ ]:
#selecting predictors of interest
cleveland_tidy <- cleveland |>
                select(age, cp, trestbps, chol, fbs, thalach, ca, num) |>
                mutate(cp = as_factor(cp),
                       trestbps = as.integer(trestbps), 
                       chol = as.integer(chol),  
                       fbs = as_factor(fbs),  
                       thalach = as.integer(thalach),
                       ca = as.integer(ca), 
                       ca = as_factor(ca),  
                       num = as_factor(num)) |>
                filter(!is.na(ca))

print("Table 1.2: Tidy dataset (excluded predictors)")
cleveland_tidy

<!-- explain NA coercion error (?) -->
_Note:_ 
> The original dataset contained missing values in the form of "?" in the __ca__ column. In order to remove them, we converted __ca__ to whole numbers using as.integer() to induce conversion of the missing values to NA so that they could be filtered out using the !is.na() function.

## 2. Preprocessing the Data

After tidying the data and selecting our predictors of interest, we preprocessed the data for our classification model by standardizing the numerical predictors and upsampling the data. First, we investigated the statistics (mean/max/min) of the numerical variables.

In [ ]:
# find the statistics (mean/max/min) of numerical variables (unstandardized)

mean_table <- cleveland_tidy |>
    mutate(mean_age = mean(age), 
          mean_trestbps = mean(trestbps), 
          mean_chol = mean(chol), 
          mean_thalach = mean(thalach)) |>
    select(mean_age, mean_trestbps, mean_chol, mean_thalach) |>
    slice(1)
max_table <- cleveland_tidy |>
    mutate(max_age = max(age), 
          max_trestbps = max(trestbps),
          max_chol = max(chol), 
          max_thalach = max(thalach)) |>
    select(max_age, max_trestbps, max_chol, max_thalach) |>
    slice(1)

min_table <- cleveland_tidy |>
    mutate(min_age = min(age), 
          min_trestbps = min(trestbps),
          min_chol = min(chol), 
          min_thalach = min(thalach)) |>
    select(min_age, min_trestbps, min_chol, min_thalach) |>
    slice(1)

print("Table 2.1: Statistics of Unstandardized Numerical Variables (age; resting blood pressure; blood cholesterol; maximum heart rate)")
print("2.1.1: Mean Stats")
mean_table
print("2.1.2: Max Stats")
max_table
print("2.1.3: Min Stats")
min_table

Investigation of the raw numerical data showed that:

* **age**: Patient ages ranged from 29 to 77 years old, and the average age was 54-55 years old.
* **trestbps**: Resting blood pressure values ranged from 94 mmHg to 200 mmHg, with an average value of about 132 mmHg. A healthy resting blood pressure value is 120 mmHg or slightly below.
* **chol**: Serum cholesterol levels ranged from 126 mg/dL to 564 mg/dL, with an average value of about 250 mg/dL. A healthy serum cholesterol level is conventionally taken to be below 200 mg/dL (see notes).
* **thalach**: Maximum heart rate values recorded upon hospital admission ranged from 71 to 202 bpm, with an average value of about 150 bpm. Healthy maximum heart rate values vary depending on the age of an individual, where an individual's maximum heart rate is expected to be 220 - the individual's age.

`Tidymodels` standardization (i.e. scaling and centering using `step_scale` and `step_center`) automatically calculates standard deviation from the mean data. However, many of the numerical predictor means were found to be far from actual "healthy" parameters. Furthermore, "healthy" values for maximum heart rate (**thalach**) vary with age. We theorized that `tidymodels` standardization (with what we have learned so far) would not provide us with information meaningful to our thesis; thus, we performed manual standardization of resting blood pressure (**trestbps**), serum cholesterol (**chol**), and maximum heart rate (**thalach**) according to literature or theoretical values. 

In [ ]:
# manual standardization of numerical variables except age (trestbps, chol, thalach)
# according to "normal" parameters (outlined above)

cleveland_thalach <- cleveland_tidy |>
                mutate(norm_thalach = (220 - age))

cleveland_tidier <- cleveland_thalach |>
                mutate(sd_thalach = ((norm_thalach - thalach)/thalach)*100,
                      sd_trestbps120 = ((trestbps - 120)/120)*100,
                      sd_chol200 = ((chol - 200)/200)*100) |>
                select(age, cp, sd_trestbps120, sd_chol200, fbs, sd_thalach, ca, num)

print("Table 2.2: Standardization of resting blood pressure, serum cholesterol, and maximum heart rate (% deviation)")
cleveland_tidier

In [ ]:
# find the statistics (mean/max/min) of standardized numerical variables

mean_table <- cleveland_tidier |>
    mutate(mean_sd_trestbps120 = mean(sd_trestbps120), 
          mean_sd_chol200 = mean(sd_chol200), 
          mean_sd_thalach = mean(sd_thalach)) |>
    select(mean_sd_trestbps120, mean_sd_chol200, mean_sd_thalach) |>
    slice(1)
max_table <- cleveland_tidier |>
    mutate(max_sd_trestbps120 = max(sd_trestbps120),
          max_sd_chol200 = max(sd_chol200), 
          max_sd_thalach = max(sd_thalach)) |>
    select(max_sd_trestbps120, max_sd_chol200, max_sd_thalach) |>
    slice(1)

min_table <- cleveland_tidier |>
    mutate(min_sd_trestbps120 = min(sd_trestbps120),
          min_sd_chol200 = min(sd_chol200), 
          min_sd_thalach = min(sd_thalach)) |>
    select(min_sd_trestbps120, min_sd_chol200, min_sd_thalach) |>
    slice(1)

print("Table 2.3: Statistics of Standardized Numerical Variables except Age (% deviation)")
print("2.3.1: Mean Stats (SD)")
mean_table
print("2.3.2: Max Stats (SD)")
max_table
print("2.3.3: Min Stats (SD)")
min_table

Investigation of the standardized numerical data 

In [ ]:
# find the number and percentage of differing presence of heart disease observations 
# in our data set to check for class imbalance

print("Table 2.4: Counts of CAD-negative vs. CAD-positive diagnoses (buff vs. sick)")
num_obs <- nrow(cleveland_tidier)
    cleveland_tidier |>
    group_by(num) |>
    summarize(
        count = n(),
        percentage = n() / num_obs * 100)


In [ ]:
set.seed(2000)

# center, scaling and balancing the heart disease data
cleve_recipe <- recipe(num ~ ., data = cleveland_tidier) |>
    # step_scale(sd_trestbps120, sd_chol200, sd_thalach) |>
    # step_center(sd_trestbps120, sd_chol200, sd_thalach) |>
    step_upsample(num, over_ratio = 1, skip = F) |>
    prep()

preprocessed_cleve <- bake(cleve_recipe, cleveland_tidier)

print("Table 2.5: Preprocessed Dataset")
preprocessed_cleve

In [ ]:
# find the number and percentage of differing presence of heart disease observations in our data set
# double check class imbalance

print("Table 2.6: Counts of preprocessed CAD-negative vs. CAD-positive diagnoses (buff vs. sick)")
num_obs <- nrow(preprocessed_cleve)
    preprocessed_cleve |>
    group_by(num) |>
    summarize(
        count = n(),
        percentage = n() / num_obs * 100)

In [ ]:
# create the TRAIN SET and TEST SET
set.seed(2000)

cleveland_split <- initial_split(preprocessed_cleve, prop = 0.75, strata = num)
cleveland_train <- training(cleveland_split) 
cleveland_test <- testing(cleveland_split)

## 3. Exploring Predictors of Interest Using the Training Set

In [ ]:
# comparing the average age for CAD-negative vs. CAD-positive patients (buff vs. sick)
exploration_tbl1 <- cleveland_train |>
    group_by(num) |>
    summarize(avg_age = mean(age)) |>
    arrange(avg_age)

print("Table 3.1: Average ages for CAD-negative vs. CAD-positive patients (buff vs. sick)")
exploration_tbl1

In [ ]:
# comparing average standard deviation of resting blood pressure, cholesterol, and 
# maximum heart rate for CAD-negative vs. CAD-positive patients (buff vs. sick)

# exploring standardized resting blood pressure (sd_trestbps120)
exploration_tbl2 <- cleveland_train |>
    group_by(num) |>
    summarize(avg_sd_trestbps120 = mean(sd_trestbps120)) |>
    arrange(avg_sd_trestbps120)

print("Table 3.2: Average standard deviation of resting blood pressure, serum cholesterol, and maximum heart rate for CAD-negative vs. CAD-positive patients (buff vs. sick)")

print("3.2.1: Average standard deviation of resting blood pressure")
exploration_tbl2

# exploring standardized cholesterol (sd_chol200)
exploration_tbl3 <- cleveland_train |>
    group_by(num) |>
    summarize(avg_sd_chol200 = mean(sd_chol200)) |>
    arrange(avg_sd_chol200)

print("3.2.2: Average standard deviation of serum cholesterol")
exploration_tbl3

# exploring standardized observed maximum heart rate (sd_thalach)
exploration_tbl4 <- cleveland_train |>
    group_by(num) |>
    summarize(avg_sd_max_heartrate = mean(sd_thalach)) |>
    arrange(avg_sd_max_heartrate)

print("3.2.3: Average standard deviation of maximum heart rate")
exploration_tbl4


In [ ]:
# comparing the counts for types of chest pain, fasting blood sugar (>/< 120 mg/dL) and 
# number of major vessels with blockage found for CAD-negative vs. CAD-positive 
# patients (buff vs. sick)

# exploring chest pain (cp) statistics
exploration_tbl5 <- cleveland_train |>
    group_by(cp) |>
    summarize(cp_count = n()) |>
    arrange(desc(cp_count))
    
print("Table 3.3: Counts for types of chest pain, fasting blood sugar (>/< 120 mg/dL) and number of major vessels with blockage found for CAD-negative vs. CAD-positive patients (buff vs. sick)")

print("3.3.1: Counts for types of chest pain (cp)")
exploration_tbl5

# exploring fasting blood sugar (fbs) statistics 
exploration_tbl6 <- cleveland_train |>
    group_by(fbs) |>
    summarize(fbs_count = n()) |>
    arrange(desc(fbs_count))

print("3.3.2: Counts for types of fasting blood sugar (fbs; fal < 120 mg/dL)")
exploration_tbl6

# exploring major vascular blockage (ca) statistics
exploration_tbl7 <- cleveland_train |>
    group_by(ca) |>
    summarize(ca_count = n()) |>
    arrange(desc(ca_count))

print("3.3.2: Counts of number of major vessels with blockage (ca; 0-3)")
exploration_tbl7

In [ ]:
# scatter plot visualizations of the relationships of numerical predictors of interest with CAD/each other

options(repr.plot.height = 4, repr.plot.width = 6)

# draw a scatter plot to visualize the relationship between serum cholesterol (standardized)
# and observed maximum heart rate (standardized)

print("Figure 1: Scatter plot visualization of numerical predictor relationships with CAD/other predictors")

exploration_plot1.1 <- cleveland_train |>
                    ggplot(aes(x = sd_chol200, y = sd_thalach, color = num)) +
                    geom_point(alpha = 0.6) +
                    geom_vline(xintercept = 0) +
                    geom_hline(yintercept = 0) +
                    labs(x = "Serum cholesterol \n(% deviation from 200 mg/dL)",
                         y = "Max heart rate \n(personal % deviation)",
                         color = "CAD diagnosis \n(neg/buff vs. \npos/sick)", 
                         title = "Fig. 1.1 \nMaximum Heart Rate vs. Serum Cholesterol \n(standardized)") + 
                    theme(text = element_text(size = 12))
exploration_plot1.1

# draw a scatter plot to visualize the relationship between serum cholesterol (standardized)
# and resting blood pressure (standardized)

exploration_plot1.2 <- cleveland_train |>
                    ggplot(aes(x = sd_chol200, y = sd_trestbps120, color = num)) +
                    geom_point(alpha = 0.6) +
                    geom_vline(xintercept = 0) +
                    geom_hline(yintercept = 0) +
                    labs(x = "Serum cholesterol \n(% deviation from 200 mg/dL)",
                         y = "Resting blood pressure \n(% deviation from 120 mmHg)",
                         color = "CAD diagnosis \n(neg/buff vs. \npos/sick)", 
                         title = "Fig. 1.2 \nResting Blood Pressure vs. Serum Cholesterol \n(standardized)") + 
                    theme(text = element_text(size = 12))
exploration_plot1.2

# draw a scatter plot to visualize the relationship between age and maximum heart rate 
# achieved (standardized) 

exploration_plot1.3 <- cleveland_train |>
                    ggplot(aes(x = age, y = sd_thalach, color = num)) +
                    geom_point(alpha = 0.6) +
                    geom_hline(yintercept = 0) +
                    geom_vline(xintercept = 54.51007, color = "blue", linetype = "dashed") + 
                    # adding vertical line at average age for whole dataset
                    labs(x = "Age (years)",
                         y = "Max heart rate \n(personal % deviation)",
                         color = "CAD diagnosis \n(neg/buff vs. \npos/sick)",
                         title = "Fig. 1.3 \nMaximum Heart Rate Achieved (standardized) vs. Age") + 
                    theme(text = element_text(size = 12))
exploration_plot1.3

# draw a scatter plot to visualize the relationship between serum cholesterol (standardized)
# and age

exploration_plot1.4 <- cleveland_train |>
                    ggplot(aes(x = age, y = sd_chol200, color = num)) +
                    geom_point(alpha = 0.6) +
                    geom_vline(xintercept = 54.51007, color = "blue", linetype = "dashed") +
                    geom_hline(yintercept = 0) +
                    labs(x = "Age (years)",
                         y = "Serum cholesterol \n(% deviation from 200 mg/dL)",
                         color = "CAD diagnosis \n(neg/buff vs. \npos/sick)", 
                         title = "Fig. 1.4 \nSerum Cholesterol (standardized) vs Age") + 
                    theme(text = element_text(size = 12))
exploration_plot1.4

# draw a scatter plot to visualize the relationship between age
# and resting blood pressure (standardized)

exploration_plot1.5 <- cleveland_train |>
                    ggplot(aes(x = age, y = sd_trestbps120, color = num)) +
                    geom_point(alpha = 0.6) +
                    geom_vline(xintercept = 54.51007, color = "blue", linetype = "dashed") +
                    geom_hline(yintercept = 0) +
                    labs(x = "Age (years)",
                         y = "Resting blood pressure \n(% deviation from 120 mmHg)",
                         color = "CAD diagnosis \n(neg/buff vs. \npos/sick)", 
                         title = "Fig. 1.5 \nResting Blood Pressure (standardized) vs Age") + 
                    theme(text = element_text(size = 12))
exploration_plot1.5

# draw a scatter plot to visualize the relationship between resting blood pressure 
# (standardized) and observed maximum heart rate (standardized)

exploration_plot1.6 <- cleveland_train |>
                    ggplot(aes(x = sd_thalach, y = sd_trestbps120, color = num)) +
                    geom_point(alpha = 0.6) +
                    geom_vline(xintercept = 0) +
                    geom_hline(yintercept = 0) +
                    labs(x = "Max heart rate \n(personal % deviation)",
                         y = "Resting blood pressure \n(% deviation from 120 mmHg)",
                         color = "CAD diagnosis \n(neg/buff vs. \npos/sick)", 
                         title = "Fig. 1.6 \nResting Blood Pressure vs. Maximum Heart Rate Achieved \n(standardized)") + 
                    theme(text = element_text(size = 12))
exploration_plot1.6


In [ ]:
# bar plot visualizations of the relationships of numerical predictors of interest with CAD/each other
# needs wrangling of categorical predictor stats

options(repr.plot.height = 4, repr.plot.width = 6)

print("Figure 2: Bar plot visualization of categorical predictor relationships with CAD/other predictors")

cp_plot2.1 <- cleveland_train |>
            select(age, cp, num)
cp_plot2.1

exploration_plot2.1 <- cp_plot2.1 |>
                    ggplot(aes(x = age, fill = as_factor(num))) +
                    geom_histogram(binwidth = 1) +
                    facet_grid(rows = vars(num)) +
                    geom_vline(xintercept = 54.51007, linetype = "dashed") +
                    labs(x = "Age (years)",
                         y = "Chest Pain Type",
                         fill = "CAD diagnosis \n(neg/buff vs. \npos/sick)",
                         title = "Fig 2.1 \n Chest Pain Type vs Age") +
                    scale_fill_manual(values = c("blue", "orange", "pink", "green")) +
                    theme(text = element_text(size = 10))
exploration_plot2.1


fbs_plot2.2 <- cleveland_train |>
            select(age, fbs, num)

exploration_plot2.2 <- ggplot(fbs_plot2.2, aes(x = age, fill = num)) +
  geom_histogram(binwidth = 1) +
  labs(x = "Age (years)",
       y = "Fasting Blood Sugar",
       fill = "CAD diagnosis \n(neg/buff vs. \npos/sick)",
       title = "Fig 2.2 \n Fasting Blood Sugar vs Age") +
  scale_fill_manual(values = c("steelblue", "darkorange")) +
  theme(text = element_text(size = 10))
exploration_plot2.2

ca_plot2.3 <- cleveland_train |>
            select(age, ca, num)

exploration_plot2.3 <- ggplot(ca_plot2.3,
                      aes(x = age,
                          fill = num)) +
  geom_histogram(binwidth = 1, position = "identity") +
  labs(x = "Age (years)",
       y = "The Number of Major Vessels Imaged By Fluoroscopy",
       fill = "CAD diagnosis \n(neg/buff vs. \npos/sick)",
       title = "Fig 2.1 \n The Number of Major Vessels Imaged By Fluoroscopy vs Age") +
  scale_fill_manual(values = c("steelblue", "darkorange")) +
  theme(text = element_text(size = 10))
exploration_plot2.3

In [ ]:
# forward selection to view predictors assisting in achieving a greater accuracy

names <- colnames(cleveland_train |> select(-num))

# create an empty tibble to store the results
accuracies <- tibble(size = integer(),
                     model_string = character(),
                     accuracy = numeric())

# create a model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = tune()) |>
     set_engine("kknn") |>
     set_mode("classification")

# create a 5-fold cross-validation object
cleveland_vfold <- vfold_cv(cleveland_train, v = 5, strata = num)

# store the total number of predictors
n_total <- length(names)

# stores selected predictors
selected <- c()

# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("num", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        cleveland_recipe <- recipe(as.formula(model_string),
                                data = cleveland_train)

        # tune the K-NN classifier with these predictors,
        # and collect the accuracy for the best K
        acc <- workflow() |>
          add_recipe(cleveland_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = cleveland_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |>
      add_row(size = i,
              model_string = models[[jstar]],
              accuracy = accs[[jstar]]) |>
      arrange(desc(accuracy))
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}

accuracies

### Predictor Exploration Analysis

Exploring the predictors 


Our tables are relevant in assisting our analysis towards answering our question, as it demonstrates the possible relationship between each predictor and our class. Where within all the tables one in particular clearly demonstrates that there are some positive relationship between the predictor age and num, the presence of heart disease.

Our plots help visually portray further relationship between more variables, whereas there is no clear relationship observed.

To conclude, within the previous tables and plots there does not seem to be any obvious relationship other than the positive relationship between age and the presence of heart disease. It is oberved that between age and the presence of heart disease, num, there is a moderately positive linear relationship where the increase of age has a correlationing increase in the presence of heart disease.

## 5. Data Analysis

In [ ]:
print("Table 5.1: Training Cleveland Data Set")
cleveland_train

In [ ]:
set.seed(2000)

# find the best k neighbour value to use with V-fold cross validation
cleveland_vfold <- vfold_cv(cleveland_train, v = 5, strata = num)

cleveland_recipe <- recipe(num ~ ., data = cleveland_train) |>
    # step_scale(sd_trestbps120, sd_chol200, sd_thalach) |>
    # step_center(sd_trestbps120, sd_chol200, sd_thalach) |>
    step_upsample(num, over_ratio = 1, skip = T) |>
    prep()

cleveland_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

gridvals = tibble(neighbors = seq(from = 1, to = 100, by = 5))

cleveland_results <- workflow() |>
            add_recipe(cleveland_recipe) |>
            add_model(cleveland_spec) |>
            tune_grid(resamples = cleveland_vfold, grid = gridvals) |>
            collect_metrics() |>
            filter(.metric == "accuracy") |>
            arrange(desc(mean))

print("Table 5.2: Metrics for Tuned Training Workflow")
cleveland_results

# plot k values against their respective accuracies and choose optimal k value
cross_val_plot <- cleveland_results |> 
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate", title = "Accuracy estimates over k-nn values") +
    theme(text = element_text(size = 12))

print("Figure 2: Finding the Best K-nn value")
cross_val_plot



In [ ]:
set.seed(2000)

# compare the accuracy of predictions to the true values in the test set
cleveland_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 11) |>
    set_engine("kknn") |>
    set_mode("classification")

cleveland_fit <- workflow() |>
  add_recipe(cleveland_recipe) |>
  add_model(cleveland_best_spec) |>
  fit(data = cleveland_train)

cleveland_predictions <- predict(cleveland_fit, cleveland_test) |> 
    bind_cols(cleveland_test)

cleveland_acc <- cleveland_predictions |> 
    metrics(truth = num, estimate = .pred_class) |> 
    select(.metric, .estimate) |> 
    head(1)
cleveland_acc

cleveland_cm <- cleveland_predictions |> 
    conf_mat(truth = num, estimate = .pred_class)
cleveland_cm

accuracy = number of correct predictions/total number of predictions *100%

precision = number of correct positive predictions/total number of positive predictions

recall = number of correct positive predictions/total number of test set observations

In [ ]:
#calculate accuracy
accuracy <- (35 + 31)/(35 + 31 + 10 + 6)

#note: we wish to predict sick observations; sick is our positive prediction
#calculate precision
precision <- 31/(31 + 6)

#calculate recall
recall <- 31/(31 + 10)

metrics <- c(accuracy, precision, recall)
metrics

# Discussion

# Work Cited

Chaudhry, R., Rahman, S., & Law, M. A. (2023, July 24). *Anatomy, thorax, heart arteries*. U.S. National Library of Medicine. https://www.ncbi.nlm.nih.gov/books/NBK470522/#:~:text=They%20are%20the%20left%20and,(left%20anterior%20descending%20artery). 

***

Marateb, H. R., & Goudarzi, S. (2015, March). *A noninvasive method for coronary artery diseases diagnosis using a clinically-interpretable fuzzy rule-based system*. Journal of Research in Medical Sciences: The Official Journal of Isfahan University of Medical Sciences. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4468223/ 

***

National Center for Chronic Disease Prevention and Health Promotion, Division for Heart Disease and Stroke Prevention. (2021, May 18). *High blood pressure symptoms and causes*. Centers for Disease Control and Prevention. https://www.cdc.gov/bloodpressure/about.htm#:~:text=%2F80%20mmHg.”-,What%20are%20normal%20blood%20pressure%20numbers%3F,less%20than%20120%2F80%20mmHg.&text=No%20matter%20your%20age%2C%20you,pressure%20in%20a%20healthy%20range. 

***

National Center for Chronic Disease Prevention and Health Promotion, Division for Heart Disease and Stroke Prevention. (2022, June 3). *Target heart rate and estimated maximum heart rate*. Centers for Disease Control and Prevention. https://www.cdc.gov/physicalactivity/basics/measuring/heartrate.htm#:~:text=You%20can%20estimate%20your%20maximum,beats%20per%20minute%20(bpm). 

***

National Center for Chronic Disease Prevention and Health Promotion, Division for Heart Disease and Stroke Prevention. (2022, December 30). *Prediabetes - your chance to prevent type 2 diabetes*. Centers for Disease Control and Prevention. https://www.cdc.gov/diabetes/basics/prediabetes.html 

***

National Center for Chronic Disease Prevention and Health Promotion, Division for Heart Disease and Stroke Prevention. (2023, February 28). *Diabetes tests*. Centers for Disease Control and Prevention. https://www.cdc.gov/diabetes/basics/getting-tested.html 

***

U.S. Department of Health and Human Services. (2023, June 30). *What is angina?*. National Heart, Lung, and Blood Institute. https://www.nhlbi.nih.gov/health/angina 

***

U.S. Department of Health and Human Services. (2023, December 20). *What is coronary heart disease?*. National Heart, Lung, and Blood Institute. https://www.nhlbi.nih.gov/health/coronary-heart-disease 

***

U.S. National Library of Medicine. (2017, September 7). *High cholesterol: Overview.* Institute for Quality and Efficiency in Health Care (IQWiG). https://www.ncbi.nlm.nih.gov/books/NBK279318/ 

***

U.S. National Library of Medicine. (2020, October 2). *Cholesterol levels: What you need to know*. MedlinePlus. https://medlineplus.gov/cholesterollevelswhatyouneedtoknow.html 